In [1]:
import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets, metrics
from sklearn.metrics import accuracy_score

In [2]:
# reading the dataset
df = pd.read_csv("data.csv")

check = np.random.rand(len(df)) < 0.8
train = df[check]
test = df[~check]

In [3]:
# choosing 12000 samples only
N1 = train.sample(12000)

train1 = train.sample(12000)
train1 = train1.replace('A', '1')
train1 = train1.replace(['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], '-1')

X = train1.drop(['class'], axis = 1).values
y = train1['class']

In [4]:
# initial model
svmc = svm.SVC(kernel = 'linear', C = 1).fit(X, y)
svmc.predict(X[0].reshape(1, -1))

array(['-1'], dtype=object)

In [5]:
# loss function as in paper
def loss_fun(f, z):
	result = 1 - int(f.predict(z.drop(['class'], axis = 1).values[0].reshape(1, -1))[0]) * int(z['class'].values)
	return result

In [6]:
# step 1: initialization
f0 = svmc
smpl = []
mplus = 0
mminus = 0
k = 0
check = 0
ztplus1 = train1.sample()

# step 6
while (mplus < (len(train1) / 2) and mminus < (len(train1) / 2)):
  # step 2
  zt = train1.sample()
  if (int(zt['class'].values) == 1):
    mplus = mplus + 1
  else:
    mminus = mminus + 1

  # step 3
  z_star = train1.sample()
  if (check == 1):
    zt = ztplus1

	# step 4
  p1 = math.exp(-loss_fun(f0, zt))
  p2 = math.exp(-loss_fun(f0, z_star))
  P = p1 / p2

  # step 5
  yt = int(zt['class'].values)
  y_star = int(z_star['class'].values)
  if (k == 5):
    ztplus1 = z_star
    check = 1
    
    P_double_dash = 1.2 * P
    if (P_double_dash > 1):
      smpl.append(z_star.index)
    else:
      ran_p = random.uniform(0, 1)
      if (P_double_dash > ran_p):
        smpl.append(z_star.index)
      
      else:
        k = 0
        
    continue
	
  if ((P == 1 and yt == -1 and y_star == -1) or (P == 1 and yt == 1 and y_star == 1) ):
    ind = z_star.index
    x = z_star.drop(['class'], axis = 1).values[0].reshape(1, -1)
    x1 = zt.drop(['class'], axis = 1).values[0].reshape(1, -1)
    P_dash = math.exp(-y_star) * int(f0.predict(x)[0]) / (-yt * int(f0.predict(x1)[0]))
    ran_p = random.uniform(0, 1)
  
    if (P_dash > ran_p):
      smpl.append(z_star.index)
      k = k + 1
    else:
      k = 0

  if ((P == 1 and yt * y_star == -1) or (P < 1)):
    ind = z_star.index
    P_dash = P
    ran_p = random.uniform(0, 1)

    if (P_dash > ran_p):
      smpl.append(z_star.index)
      k = k + 1
    else:
      k = 0

In [7]:
# finding the further to be processed samples
further = []
for i in range(len(smpl)):
  further.append(int(smpl[i][0]))

In [8]:
final_data = df.iloc[further]
print(final_data.head())

      class  f1  f2  f3  f4  f5  f6  f7  ...  f9  f10  f11  f12  f13  f14  f15  f16
4279      T   1   0   1   0   0   7  14  ...   4    7   10    8    0    8    0    8
5860      G   5  10   7   8   8   9   6  ...   2    6    6   10    8    8    5   10
15011     O   5  10   7   8   3   9   6  ...   8    7    5   10    3    8    4    8
1680      F   4  10   6   8   3   4  13  ...   5   12   10    3    2   10    2    4
3588      B   4   6   6   4   7   8   7  ...   2    6    7    8    5   10    9    8

[5 rows x 17 columns]


In [9]:
# prediction using SVMC
X1 = final_data.drop(['class'], axis = 1).values
y1 = final_data['class']

new_svmc = svm.SVC(kernel = 'linear', C = 1).fit(X1, y1)

X = df.drop(['class'], axis = 1).values
y = df['class']

y_pred = new_svmc.predict(X)

print("Accuracy of the model: ", accuracy_score(y, y_pred))

Accuracy of the model:  0.8475
